In [ ]:
# default_exp treebaltimore

# Guided Walkthrough

Todo: 
- Refractor Queries for no Point In Polygons
- Wrap as Function

## Whats Inside?: 

#### __Indicators Used__

- ❌ 199 - __TreePlnt__ - (TreeBaltimore) Measures the number of trees planted annually by the TreeBaltimore program.

SQL had a problem

Working Query. Original had 'quantity' but 2017 works with 'qty'

with tbl AS (
 select (sum(qty)::real) as result, csa
  from vital_signs.match_csas_and_bc_by_geom('sustainability.trees_2016', 'gid', 'the_geom') a
  left join sustainability.trees_2016 b on a.gid = b.gid
    group by csa
 )

select * from tbl where 1 = 1 ORDER BY csa ASC;
 update vital_signs.data
 set trees = result from tbl where data.csa = tbl.csa and data_year = '2016';

 Working Query. Original had 'quantity' but 2017 works with 'f__of_tree'

## SETUP:

### Import Modules

In [ ]:
# @title Run: Install Modules

In [ ]:
%%capture
! pip install geopy
! pip install geopandas
! pip install geoplot
! pip install dataplay

In [ ]:
!pip install dexplot

In [ ]:
%%capture
# export
# @title Run: Import Modules

# These imports will handle everything
import os
import sys
import csv
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim
import folium
from folium import plugins

from dataplay.merge import mergeDatasets 

import dexplot as dxp

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

### Configure Enviornment

In [ ]:
# This will just beautify the output

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.precision', 2)
# pd.reset_option('max_colwidth')
pd.set_option('max_colwidth', 50)
# pd.reset_option('max_colwidth')

In [ ]:
#hide
# %matplotlib inline
# !jupyter nbextension enable --py widgetsnbextension

## Read in df

In [ ]:
csa_gdf = gpd.read_file("https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson")
csa_gdf.plot(column='tpop10')

In [ ]:
csa_gdf.head()

### (Optional) GoogleDrive Access

In [ ]:
# (Optional) Run this cell to gain access to Google Drive (Colabs only) 
from google.colab import drive

# Colabs operates in a virtualized enviornment
# Colabs default directory is at ~/content.
# We mount Drive into a temporary folder at '~/content/drive' 

drive.mount('/content/drive')

In [ ]:
cd drive/'MyDrive'/vitalSigns

In [ ]:
ls

In [ ]:
cd treeplnt

In [ ]:
crs = {'init' :'epsg:2248'} 
gdf = gpd.read_file("All2019PlantingData.shp", crs=crs)
geometry=[Point(xy) for xy in zip(gdf['Longitude'], gdf['Latitude'])]
gdf = gpd.GeoDataFrame( gdf, geometry=geometry)
gdf.plot()
gdf.head()

In [ ]:
from dataplay import geoms 

In [ ]:
help(geoms)

In [ ]:
#export 
fdf = geoms.workWithGeometryData(method='pinp', df=gdf, polys=csa_gdf, ptsCoordCol='geometry', polygonsCoordCol='geometry', polyColorCol=False, polygonsLabel='RecordsInCsa', pntsClr='red', polysClr='white')
findf = fdf.copy()
findf = findf[['CSA2010', 'tpop10', 'pointsinpolygon']]

findf = findf.append({
    'CSA2010': 'Baltimore City', 
    'tpop10': findf['tpop10'].sum(), 
    'pointsinpolygon': findf['pointsinpolygon'].sum(),
     }, ignore_index=True)

In [ ]:
findf.to_csv('treeplnt19.csv')

In [ ]:
findf.dtypes

In [ ]:
findf.tail()